# Quickstart

These instructions will help you create a report in <1 minute. We will build a simple HTML report using Python based on the [Iris dataset](https://en.wikipedia.org/wiki/Iris_flower_data_set).

## Installation

First, install Datapane using `pip` or `conda`. Check out our [installation page](/install-datapane/) for installation options. 

!!! note
    You can also download this Quickstart as a [Jupyter Notebook](https://raw.githubusercontent.com/datapane/datapane/main/public-docs/docs/quickstart.ipynb){:download=quickstart.ipynb}.


```python
pip3 install -U datapane
```

## Setting things up

In [ ]:
import altair as alt
import datapane as dp
from vega_datasets import data

We've imported `datapane`, the popular visualization library `altair`, and `vega_datasets` which contains some sample datasets.

Let's load the Iris dataset and get a list of the features.

In [ ]:
df = data.iris()
columns = list(df.columns)
print(columns)

## Blocks

Your report is comprised of _Blocks_, which are Python objects that wrap around common objects such as datasets, plots, and other blocks. There are display blocks, such as _Plot_ or _DataTable_, and layout blocks, such as _Select_ and _Group_.

Having loaded our DataFrame above and with knowledge of our column names, we first create a simple scatterplot using the Altair plotting library. 

We then build a simple set of blocks which presents two tabs: one with our plot, and one with our DataFrame.

In [ ]:
fig = (
    alt.Chart(df)
    .mark_point()
    .encode(x=alt.X("sepalLength", scale=alt.Scale(zero=False)), 
            y=alt.X("sepalWidth", scale=alt.Scale(zero=False)),
            color="species")
)

view = dp.Select(dp.Plot(fig, label="Plot"), dp.DataTable(df, label="Data"))
view

## Reports

Once we have a view, we can save it as an HTML report to share.

Let's save it as a report and [open it in a new window](./quickstart_report.html){:target=_blank}.

In [ ]:
dp.save_report(view, "quickstart_report.html", open=True)

## Sharing your report

That's it! As Datapane has created a standalone HTML file, you can now share this Slack or email without deploying any code or configuring a backend server. 